# Custom exploration of Eniac's database

In [1]:
import pandas as pd

In [42]:
orderlines = pd.read_csv('data/eniac/orderlines.csv')
products = pd.read_csv('data/eniac/products.csv')
orders = pd.read_csv('data/eniac/orders.csv')

# Orderlines info

In [4]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [46]:
orderlines.head(10)

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27
6,1119115,299544,0,1,APP1582,1137.99,2017-01-01 01:17:21
7,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16
8,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34
9,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11


# Products info

In [10]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


## Orders info

In [13]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


## How many orders are there?

In [8]:
orderlines.id_order.nunique()

204855

## How many products are there?

In [11]:
products.sku.nunique()

10579

## What period of time do these orders comprise?

In [18]:
orders.created_date.agg(['min', 'max'])

min    2017-01-01 00:07:19
max    2018-03-14 13:58:36
Name: created_date, dtype: object

## How many orders are Completed?

In [19]:
orders.state.unique()

array(['Cancelled', 'Completed', 'Pending', 'Shopping Basket',
       'Place Order'], dtype=object)

In [23]:
orders.loc[orders.state == 'Completed'].state.value_counts()

Completed    46605
Name: state, dtype: int64

## How should revenue be computed?

First, I have to convert 'unit_price' to Float

In [43]:
orderlines['unit_price'] = orderlines['unit_price'].str.rsplit('.',1).apply(lambda x : x[0]+ ',' + x[1])
orderlines['unit_price'] = orderlines['unit_price'].str.replace('.','',regex=True)
orderlines['unit_price'] = orderlines['unit_price'].str.replace(',','.',regex=True)
orderlines['unit_price'] = orderlines['unit_price'].astype(float)

Revenue for entire database

In [44]:
orderlines.assign(total_price = orderlines.unit_price * orderlines.product_quantity)['total_price'].sum()

128776222.02999999

To use date column as datetime:

In [49]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

Revenue for each year:

In [52]:
orderlines.assign(total_price = orderlines.unit_price * orderlines.product_quantity, year = orderlines.date.dt.strftime('%Y')).groupby('year').agg('sum')

,id,id_order,product_id,product_quantity,unit_price,total_price
year,,,,,,
2017,292571254920,86093077640,0,248816,83522983.71,90144159.46
2018,118392903252,37379522639,0,80776,36967475.10,38632062.57
